# 1NN против RandomForest


В этом задании будет использоваться датасет `digits` из `sklearn.datasets`. Оставьте _последние_ 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами `1NN` с `RandomForestClassifier` из `sklearn` на 1000 деревьях.

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка $N log N$ сравнений (более строго говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. 

За выбор метода поиска ближайших соседей в `KNeigborsClassifier` из `sklearn` отвечает параметр `algorithm` — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных `ball tree` и `kd tree`.

Доля ошибок, допускаемых 1NN на тестовой выборке, — **ответ в задании 1**.

## Задание 2

Теперь обучите на обучающей выборке `RandomForestClassifier(n_estimators=1000)` из `sklearn`. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — **ответ в задании 2**. 

Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — `1NN`. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [2]:
def write_answer(*answers, task_number=None):
    filename = f'task{task_number}.txt' if task_number else 'task.txt'
    with open(filename, 'w') as f:
        f.write(' '.join([str(ans) for ans in answers]))

In [3]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [4]:
digits = load_digits()
X, y = digits.data, digits.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False, random_state=0)

In [6]:
def squared_euclid(point1, point2):
    return sum([(coord1-coord2)**2 for coord1, coord2 in zip(point1, point2)])

In [7]:
y_predicted = []

In [8]:
for test_point in X_test:
    closest_neighbors = [(squared_euclid(test_point, train_point), train_label) for train_point, train_label 
                          in zip(X_train, y_train)]
    closest_neighbors.sort()
    y_predicted.append(closest_neighbors[0][1])

In [9]:
knn_accuracy = accuracy_score(y_test, y_predicted)

In [10]:
write_answer(1-knn_accuracy, task_number=1)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
rf_classifier = RandomForestClassifier(n_estimators=100)

In [13]:
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
rfc_accuracy = accuracy_score(y_test, rf_classifier.predict(X_test))

In [16]:
write_answer(1-rfc_accuracy, task_number=2)